
# Classification Evaluation Metrics


In [1]:

fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime

import random


In [5]:

DEBUG = True
seed = 1919


In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

#preds_dataset_name = 'predictions_dataset_2022-10-18.csv'
preds_dataset_name = 'predictions_dataset_2022-10-26.csv'

df_eval_raw = pd.read_csv(f'{fileloc}/{preds_dataset_name}')

print(df_eval_raw.shape)
print()
print(df_eval_raw.columns.values.tolist())


(571461, 18)

['@id_DOG', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'x_dist', 'x_races_tot', 'position', 'ft_sp', 'x_no_prev_races', 'x_avg_speed_prev', 'yearqtr', 'ttflag', 'preds']


In [8]:

df_num_dogs = df_eval_raw.groupby("@id_RACE").agg( { '@id_DOG':'nunique' } ).reset_index().rename(columns = {'@id_DOG':'x_num_dogs'})

df_evaluation = pd.merge(df_eval_raw, df_num_dogs, on = ["@id_RACE"], how = 'left')

print(df_evaluation.x_num_dogs.value_counts())
    

8    312000
7    136031
6     88374
5     28265
4      5432
3       924
2       284
1       151
Name: x_num_dogs, dtype: int64


In [9]:

df_evaluation.isna().sum()


@id_DOG                0
DogName                0
y_runtime              0
TrainerName            0
@id_RACE               0
RaceNum                0
RaceName              15
Track                  0
Event_Dt               0
x_dist                 0
x_races_tot            0
position               0
ft_sp               2934
x_no_prev_races        0
x_avg_speed_prev       0
yearqtr                0
ttflag                 0
preds                  0
x_num_dogs             0
dtype: int64

In [11]:

##
groupings = ['@id_RACE']

# Actual Result
df_evaluation['Top3_actual'] = df_evaluation.apply(lambda x : 1 if x.position <=3 else 0, axis = 1)

print(datetime.now())


2022-10-26 11:45:33.952477


In [12]:

# Random Variable in Dataset

import random
#print(random.uniform(0, 1))

df_evaluation['rand_mean'] = 0.5
df_evaluation['rand_uniform'] = np.random.normal(df_evaluation['rand_mean'])

## Rank from the random variables 
df_evaluation["position_random"] = df_evaluation.groupby(groupings)["rand_uniform"].rank("dense", ascending=False)

print(datetime.now())


2022-10-26 11:45:34.687265


In [13]:

df_evaluation["position_random"].value_counts()


1.0    80774
2.0    80623
3.0    80481
4.0    80173
5.0    78815
6.0    73162
7.0    58433
8.0    39000
Name: position_random, dtype: int64

In [14]:

## Rank for Place Preds
df_evaluation["position_fasttrack"] = df_evaluation.groupby(groupings)["ft_sp"].rank("dense", ascending=True)

# Winner / Place Columns
df_evaluation['Winner_fastrack'] = df_evaluation.apply(lambda x : 1 if x.position_fasttrack <= 1 else 0, axis = 1)
df_evaluation['Top3_fastrack'] = df_evaluation.apply(lambda x : 1 if x.position_fasttrack <= 3 else 0, axis = 1)

print(datetime.now())


2022-10-26 11:45:49.701227


In [15]:

## Rank for Place Preds
df_evaluation["position_preds"] = df_evaluation.groupby( groupings )["preds"].rank("dense", ascending=False)

# Winner / Place Preds Columns
df_evaluation['Winner_preds'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 1 else 0, axis = 1)
df_evaluation['Top3_preds'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)

print(datetime.now())


2022-10-26 11:46:03.568304


In [16]:

print(df_evaluation.yearqtr.value_counts())


202103    85161
202104    84777
202203    84475
202102    81430
202202    81113
202201    77794
202101    76711
Name: yearqtr, dtype: int64


In [17]:

df_evaluation.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
@id_DOG,571461.0,4.529896e+08,1.122953e+08,1.083910e+08,3.836047e+08,4.517682e+08,5.283827e+08,2.145360e+09
y_runtime,571461.0,2.444311e+01,4.980097e+00,1.000000e+01,2.058000e+01,2.354000e+01,2.726000e+01,3.961200e+02
@id_RACE,571461.0,7.291770e+08,5.831171e+07,6.238619e+08,6.798781e+08,7.290297e+08,7.792435e+08,8.292517e+08
RaceNum,571461.0,6.131724e+00,3.286222e+00,1.000000e+00,3.000000e+00,6.000000e+00,9.000000e+00,1.800000e+01
x_dist,571461.0,4.175169e+02,8.384859e+01,2.590000e+02,3.500000e+02,4.000000e+02,4.610000e+02,7.320000e+02
x_races_tot,571461.0,1.862874e+01,1.608810e+01,0.000000e+00,6.000000e+00,1.400000e+01,2.700000e+01,1.260000e+02
position,571461.0,4.124696e+00,2.150335e+00,1.000000e+00,2.000000e+00,4.000000e+00,6.000000e+00,8.000000e+00
ft_sp,568527.0,1.690650e+01,2.097902e+01,2.600000e-01,4.200000e+00,8.700000e+00,2.050000e+01,9.999900e+02
x_no_prev_races,571461.0,5.879666e-04,2.424091e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
x_avg_speed_prev,571461.0,1.709293e+04,3.319987e+02,1.138747e+04,1.691188e+04,1.711551e+04,1.729405e+04,4.020000e+04


In [18]:

df_evaluation.sort_values(by = ['@id_RACE','position']).head()


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,Event_Dt,x_dist,x_races_tot,position,ft_sp,x_no_prev_races,x_avg_speed_prev,yearqtr,ttflag,preds,x_num_dogs,Top3_actual,rand_mean,rand_uniform,position_random,position_fasttrack,Winner_fastrack,Top3_fastrack,position_preds,Winner_preds,Top3_preds
196119,434274825,FERDINAND BOY,24.98,D Geall,623861932,6,GRV VIC BRED SERIES FINAL (VICGREYS),Ballarat,2021-01-01,450,0,1,4.2,0,17461.106781,202101,Train,26.515457,8,1,0.5,1.335867,2.0,3.0,0,1,6.0,0,0
125155,380917315,BAD GUY,25.03,J Thompson,623861932,6,GRV VIC BRED SERIES FINAL (VICGREYS),Ballarat,2021-01-01,450,0,2,5.9,0,17096.418815,202101,Train,26.619202,8,1,0.5,0.156813,5.0,4.0,0,0,2.0,0,1
274144,465017181,MASKED AVENGER,25.08,J Borg,623861932,6,GRV VIC BRED SERIES FINAL (VICGREYS),Ballarat,2021-01-01,450,0,3,3.2,0,17484.535259,202101,Train,26.508791,8,1,0.5,2.122479,1.0,1.0,1,1,8.0,0,0
56060,318755759,ASTON CORINNA,25.32,A Dailly,623861932,6,GRV VIC BRED SERIES FINAL (VICGREYS),Ballarat,2021-01-01,450,0,4,16.1,0,17189.363691,202101,Train,26.592760,8,0,0.5,-0.202349,6.0,5.0,0,0,4.0,0,0
131141,387031520,GYPSY ZELDA,25.45,A Dailly,623861932,6,GRV VIC BRED SERIES FINAL (VICGREYS),Ballarat,2021-01-01,450,0,5,32.1,0,17460.580911,202101,Train,26.515604,8,0,0.5,-0.738278,7.0,8.0,0,0,5.0,0,0


In [19]:

pd.crosstab(df_evaluation.position, df_evaluation.position_preds)


position_preds,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
position,,,,,,,,
1,6533,8122,9636,11239,12848,13268,11218,8063
2,7768,9559,10981,12207,12658,11948,9393,6161
3,9374,10890,11678,12059,11854,10966,8425,5261
4,11054,11802,12063,11719,11237,10042,7472,4755
5,12692,12443,11879,11206,10403,9134,6873,4159
6,13266,12002,10626,9886,9102,8133,6210,3883
7,11305,9465,8359,7360,6717,6101,5545,3500
8,8798,6370,5287,4523,4023,3567,3264,3127


In [20]:

df_evaluation.groupby( [ 'ttflag', 'Top3_actual', 'Top3_preds']).agg({'@id_DOG':'count'}).reset_index()


,ttflag,Top3_actual,Top3_preds,@id_DOG
0,Test,0,0,51153
1,Test,0,1,44468
2,Test,1,0,44521
3,Test,1,1,25446
4,Train,0,0,120788
5,Train,0,1,112943
6,Train,1,0,113047
7,Train,1,1,59095


In [21]:

pd.crosstab(df_evaluation.Top3_actual, df_evaluation.Top3_preds, normalize='all')


Top3_preds,0,1
Top3_actual,,
0,0.300880,0.275454
1,0.275728,0.147938


In [22]:

df_evaluation.head()


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,Event_Dt,x_dist,x_races_tot,position,ft_sp,x_no_prev_races,x_avg_speed_prev,yearqtr,ttflag,preds,x_num_dogs,Top3_actual,rand_mean,rand_uniform,position_random,position_fasttrack,Winner_fastrack,Top3_fastrack,position_preds,Winner_preds,Top3_preds
0,108390981,DOLLY DOES IT,26.43,M Elphinstone,626271182,10,GREAT NORTHERN,Devonport,2021-01-05,452,0,4,11.0,1,17008.287271,202101,Train,26.851376,8,0,0.5,-0.832116,7.0,5.0,0,0,1.0,1,1
1,108390981,DOLLY DOES IT,26.40,M Elphinstone,629797242,10,GREAT NORTHERN,Devonport,2021-01-12,452,1,5,7.0,0,17101.778282,202101,Train,26.731966,7,0,0.5,2.058338,1.0,5.0,0,0,4.0,0,0
2,108390981,DOLLY DOES IT,26.20,M Elphinstone,629797513,4,SUREPICK,Devonport,2021-01-19,452,2,1,7.0,0,17111.495202,202101,Train,26.727310,7,1,0.5,-0.378054,6.0,4.0,0,0,6.0,0,0
3,108390981,DOLLY DOES IT,26.48,M Elphinstone,638810808,8,LADBROKES,Devonport,2021-02-16,452,3,7,7.0,0,17158.299600,202101,Train,26.712110,8,0,0.5,-0.488168,7.0,4.0,0,0,6.0,0,0
4,108390981,DOLLY DOES IT,26.91,M Elphinstone,643554706,8,NEW RECRUIT 2012,Devonport,2021-03-02,452,4,8,91.0,0,17136.096301,202101,Train,26.716540,8,0,0.5,0.408867,3.0,8.0,0,0,3.0,0,1


In [23]:

print(datetime.now())


2022-10-26 11:46:04.707760


In [24]:

# Random


In [25]:


# Bet on all Preds = s1
df_evaluation['rand_1'] = df_evaluation.apply( lambda x : True if x.position_random == 1 else False, axis = 1 )
df_evaluation['h_rand_1'] = df_evaluation.apply( lambda x : 1 if ( x.rand_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.rand_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_evaluation['p_rand_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.rand_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.rand_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["rand_all_plc"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_random) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_random <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_random <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)
df_evaluation['h_rand_all_plc'] = df_evaluation.apply( lambda x : 1 if ( x.rand_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.rand_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_evaluation['p_rand_all_plc'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.rand_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.rand_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()

print(datetime.now())



2022-10-26 11:47:50.573997


In [26]:

# Benchmark


In [27]:

# Bet on all Preds = s1
df_evaluation['ft_1'] = df_evaluation.apply( lambda x : True if x.position_fasttrack == 1 else False, axis = 1 )
df_evaluation['h_ft_1'] = df_evaluation.apply( lambda x : 1 if ( x.ft_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.ft_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_evaluation['p_ft_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.ft_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.ft_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["ft_all_plc"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_fasttrack) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_fasttrack <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_fasttrack <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)
df_evaluation['h_ft_all_plc'] = df_evaluation.apply( lambda x : 1 if ( x.ft_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.ft_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_evaluation['p_ft_all_plc'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.ft_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.ft_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()

print(datetime.now())
#3Min runtime



2022-10-26 11:49:31.858357


In [28]:

# Strategies


In [29]:

# Bet on all Preds = s1
df_evaluation['s_1'] = df_evaluation.apply( lambda x : True if x.position_preds == 1 else False, axis = 1 )
df_evaluation['h_s_1'] = df_evaluation.apply( lambda x : 1 if ( x.s_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.s_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_evaluation['p_s_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.s_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.s_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["s_all_plc"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_preds) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_preds <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_preds <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)

df_evaluation['h_s_all_plc'] = df_evaluation.apply( lambda x : 1 if ( x.s_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.s_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_evaluation['p_s_all_plc'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.s_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.s_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()

print(datetime.now())
#3min runtime



2022-10-26 11:51:15.270291


In [30]:

df_evaluation.head()


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,Event_Dt,x_dist,x_races_tot,position,ft_sp,x_no_prev_races,x_avg_speed_prev,yearqtr,ttflag,preds,x_num_dogs,Top3_actual,rand_mean,rand_uniform,position_random,position_fasttrack,Winner_fastrack,Top3_fastrack,position_preds,Winner_preds,Top3_preds,rand_1,h_rand_1,p_rand_1,rand_all_plc,h_rand_all_plc,p_rand_all_plc,ft_1,h_ft_1,p_ft_1,ft_all_plc,h_ft_all_plc,p_ft_all_plc,s_1,h_s_1,p_s_1,s_all_plc,h_s_all_plc,p_s_all_plc
0,108390981,DOLLY DOES IT,26.43,M Elphinstone,626271182,10,GREAT NORTHERN,Devonport,2021-01-05,452,0,4,11.0,1,17008.287271,202101,Train,26.851376,8,0,0.5,-0.832116,7.0,5.0,0,0,1.0,1,1,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,True,0,-1.0,True,0,-1.0
1,108390981,DOLLY DOES IT,26.40,M Elphinstone,629797242,10,GREAT NORTHERN,Devonport,2021-01-12,452,1,5,7.0,0,17101.778282,202101,Train,26.731966,7,0,0.5,2.058338,1.0,5.0,0,0,4.0,0,0,True,0,-1.0,True,0,-1.0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0
2,108390981,DOLLY DOES IT,26.20,M Elphinstone,629797513,4,SUREPICK,Devonport,2021-01-19,452,2,1,7.0,0,17111.495202,202101,Train,26.727310,7,1,0.5,-0.378054,6.0,4.0,0,0,6.0,0,0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0
3,108390981,DOLLY DOES IT,26.48,M Elphinstone,638810808,8,LADBROKES,Devonport,2021-02-16,452,3,7,7.0,0,17158.299600,202101,Train,26.712110,8,0,0.5,-0.488168,7.0,4.0,0,0,6.0,0,0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0
4,108390981,DOLLY DOES IT,26.91,M Elphinstone,643554706,8,NEW RECRUIT 2012,Devonport,2021-03-02,452,4,8,91.0,0,17136.096301,202101,Train,26.716540,8,0,0.5,0.408867,3.0,8.0,0,0,3.0,0,1,False,0,0.0,True,0,-1.0,False,0,0.0,False,0,0.0,False,0,0.0,True,0,-1.0


In [31]:

pd.set_option('display.max_rows', 50)


In [32]:

# Correlations


In [33]:

print('Position vs Random')
print(df_evaluation['position'].corr( df_evaluation['position_random'] ))
print()

print('Position vs Preds')
print(df_evaluation['position'].corr( df_evaluation['position_preds'] ))
print()

print('Position vs FastTrack')
print(df_evaluation['position'].corr( df_evaluation['position_fasttrack'] ))

print()
print()
print('Preds vs Fasttrack')
print(df_evaluation['position_preds'].corr( df_evaluation['position_fasttrack'] ))


Position vs Random
0.05204491596260066

Position vs Preds
-0.15005133147605046

Position vs FastTrack
0.4633801067726823


Preds vs Fasttrack
-0.27834366874178745


In [34]:

# Classic overestimation of Placement problems -- placing top 3 for 8, and also 3 for 7,6,5,4,3,2,1 racers (fix)
# Classic using WIN price for PLACE issue

df_eval = df_evaluation.groupby(['ttflag'])\
.agg({'ft_1':'sum', 'h_ft_1':'sum', 'p_ft_1':'sum', 'ft_all_plc':'sum', 'h_ft_all_plc':'sum', 'p_ft_all_plc':'sum'\
      , 's_1':'sum', 'h_s_1':'sum', 'p_s_1' :'sum' ,'s_all_plc':'sum', 'h_s_all_plc':'sum', 'p_s_all_plc':'sum'\
      , 'rand_1':'sum', 'h_rand_1':'sum', 'p_rand_1' :'sum' ,'rand_all_plc':'sum', 'h_rand_all_plc':'sum', 'p_rand_all_plc':'sum'\
      ,'@id_RACE':'nunique' })\
.reset_index()\
.transpose()


In [35]:

df_eval


,0,1
ttflag,Test,Train
ft_1,24033,59267
h_ft_1,9856,24123
p_ft_1,-3088.19,-8040.33
ft_all_plc,59375,145292
h_ft_all_plc,16451,40237
p_ft_all_plc,-9618.27,-22552
s_1,23360,57430
h_s_1,2142,4391
p_s_1,-7428.37,-19326.1


In [36]:

#df_eval.to_csv(f'{fileloc}/evaluation_2022-10-18_extended_totals.csv', index = True)
df_eval.to_csv(f'{fileloc}/evaluation_2022-10-26_extended_totals.csv', index = True)


In [37]:

df_res = df_evaluation.groupby(['ttflag','Track'])\
.agg({'ft_1':'sum', 'h_ft_1':'sum', 'p_ft_1':'sum', 'ft_all_plc':'sum', 'h_ft_all_plc':'sum', 'p_ft_all_plc':'sum'\
      , 's_1':'sum', 'h_s_1':'sum', 'p_s_1' :'sum' ,'s_all_plc':'sum', 'h_s_all_plc':'sum', 'p_s_all_plc':'sum'\
      , 'rand_1':'sum', 'h_rand_1':'sum', 'p_rand_1' :'sum' ,'rand_all_plc':'sum', 'h_rand_all_plc':'sum', 'p_rand_all_plc':'sum'\
      ,'@id_RACE':'nunique' })\
.reset_index()\
.transpose()


In [38]:
df_res

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110
ttflag,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,...,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train
Track,Albion Park,Angle Park,Ballarat,Bathurst,Bendigo,Broken Hill,Bulli,Bundaberg,Canberra,Cannington,Capalaba,Casino,Coonamble,Dapto,Darwin,Dubbo,Gawler,Geelong,Gosford,Goulburn,Grafton,Gunnedah,Healesville,Hobart,Horsham,...,Mandurah,Meadows,Moree,Mount Gambier,Murray Bridge,Muswellbrook,Nowra,Potts Park,Richmond,Rockhampton,Sale,Sandown Park,Shepparton,Tamworth,Taree,Temora,The Gardens,Townsville,Traralgon,Wagga,Warragul,Warrnambool,Wauchope,Wentworth Park,Young
ft_1,964,773,747,284,715,110,254,159,0,1015,422,483,65,268,362,271,632,705,331,314,419,301,721,267,393,...,2559,1518,119,1457,1620,83,591,87,2536,687,1195,1435,1774,184,453,430,1618,1347,226,582,2010,1544,158,1169,141
h_ft_1,407,326,298,133,275,47,107,76,0,396,212,187,20,101,153,111,270,301,134,139,177,120,285,102,174,...,1016,622,32,623,768,28,255,24,1050,270,496,560,687,73,164,176,645,505,96,225,832,601,62,465,32
p_ft_1,-50.91,-103.9,-97.76,-7.38,-134.1,-21.18,-49.85,3,0,-78.62,-14.88,-65.22,-28.55,-48.28,-46,-54.58,-98.11,-57.32,-35.81,-43.8,-51.42,-44.71,-133.48,-52.41,-62.18,...,-309.96,-194.68,-66.2,-143.89,-185.49,-25,-56.95,-37.55,-276.45,-98.7,-140.86,-227.34,-301.96,-31.5,-76.88,-69.33,-181.31,-194.21,-19.5,-52.34,-223.88,-233.72,-18.9,-149.82,-68.4
ft_all_plc,2576,1678,1897,689,1810,231,581,399,0,2566,1122,1290,148,668,818,681,1369,1672,844,779,1044,817,1808,733,912,...,6751,3549,188,2982,3278,204,1363,209,6846,1875,2762,3426,4495,453,1201,1105,4337,3498,512,1528,4922,3824,395,2979,377
h_ft_all_plc,692,498,509,198,489,65,180,120,0,679,328,342,38,183,226,175,423,476,227,227,307,205,507,183,274,...,1799,985,44,927,1078,51,405,46,1834,502,782,984,1194,122,287,292,1125,896,146,395,1389,1081,108,809,74
p_ft_all_plc,-391.24,-227,-314.36,-144.78,-264.4,-58.38,-98.25,-39.75,0,-342.06,-256.48,-207.77,-45.35,-84.71,-163.5,-193.83,-208.66,-223.06,-156.86,-132.75,-142.22,-193.16,-251.48,-163.91,-161.18,...,-940.06,-630.3,-89.5,-419.14,-480.58,-57.5,-192,-86.35,-1144.17,-263.13,-406.3,-281.24,-723.96,-141.1,-271.78,-209.53,-827.65,-619.33,-94.1,-218.82,-644.02,-382.12,-37.05,-362.82,-171.8
s_1,944,773,732,273,705,100,248,155,1,1063,413,473,61,262,350,264,621,690,324,304,409,296,704,259,388,...,2641,1478,67,1417,1584,71,581,74,2476,670,1168,1413,1737,164,438,418,1573,1313,215,569,1975,1508,153,1134,105
h_s_1,86,106,56,19,57,10,16,11,1,119,34,31,4,22,30,18,67,76,35,19,29,24,49,27,27,...,236,123,4,125,155,6,42,7,167,40,86,112,119,12,32,24,128,84,4,34,141,103,5,91,7


In [39]:

#df_res.to_csv(f'{fileloc}/evaluation_2022-10-18_extended_bytrack.csv', index = True)
df_res.to_csv(f'{fileloc}/evaluation_2022-10-26_extended_bytrack.csv', index = True)


In [40]:

print("THE END")


THE END
